# Fully Connected Naural Networks - Regularization/L2 - No Convolutions

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

First reload the data we generated in __notMNIST_nonTensorFlow_comparisons.ipynb__.

In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:

* data as a flat matrix,
* labels as float 1-hot encodings.

In [3]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 1 to [0.0, 1.0, 0.0 ...], 2 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [4]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

## L2 regularization
L2 amounts to adding a penalty on the norm of the weights to the loss. 
In TensorFlow, you can compute the L2 loss for a tensor t using nn.l2_loss(t). 

### Logistic Model 

In [5]:
image_size=28
##### logistic model 
def create_log_model_and_run(graph,
                         train_dataset,
                         train_labels,
                         valid_dataset,
                         valid_labels,
                         test_dataset,
                         test_labels,
                         beta,
                         num_steps,
                         num_labels=10,batch_size = 128):
    
    with graph.as_default():
      # Input data. For the training data, we use a placeholder that will be fed
      # at run time with a training minibatch.
      tf_train_dataset = tf.placeholder(tf.float32,shape=(batch_size, image_size * image_size))
      tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
        
      tf_valid_dataset = tf.constant(valid_dataset)
      tf_test_dataset = tf.constant(test_dataset)

      # Variables.
      weights = tf.Variable(tf.truncated_normal([image_size * image_size, num_labels]))
      biases = tf.Variable(tf.zeros([num_labels]))

      # Training computation.
      logits = tf.matmul(tf_train_dataset, weights) + biases
      loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))+(1/batch_size)*beta*tf.nn.l2_loss(weights)

      # Optimizer.
      global_step = tf.Variable(0)  # count the number of steps taken.
      learning_rate = tf.train.exponential_decay(0.5, global_step, 100000, 0.96, staircase=True)
      optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
      #optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)

      # Predictions for the training, validation, and test data.
      train_prediction = tf.nn.softmax(logits)
      
      valid_prediction = tf.nn.softmax(tf.matmul(tf_valid_dataset, weights) + biases)
      test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)
        
    test_accuracy = 0
    with tf.Session(graph=graph) as session:
      tf.global_variables_initializer().run()
      print("Initialized")
      for step in range(num_steps):
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        # Generate a minibatch.
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run(
          [optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 500 == 0):
          print("Minibatch loss at step %d: %f" % (step, l))
          print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
          print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
          test_accuracy = accuracy(test_prediction.eval(), test_labels)
          print("Test accuracy: %.1f%%" % test_accuracy)
    
    return test_accuracy
    

In [6]:
num_steps = 3001

betas = [0, 0.001,0.01,0.1,1,10]
test_accuracy = np.zeros(len(betas))
i = 0
for beta in betas:
  print("\n>>>>>>>>>> Beta: %f%%" % beta)
  graph = tf.Graph()
  test_accuracy[i] = create_log_model_and_run(graph,
                         train_dataset,
                         train_labels,
                         valid_dataset,
                         valid_labels,
                         test_dataset,
                         test_labels,
                         beta,
                         num_steps)
   
  i = i + 1


>>>>>>>>>> Beta: 0.000000%
Initialized
Minibatch loss at step 0: 18.741056
Minibatch accuracy: 10.2%
Validation accuracy: 10.7%
Test accuracy: 10.4%
Minibatch loss at step 500: 1.130287
Minibatch accuracy: 78.9%
Validation accuracy: 75.8%
Test accuracy: 83.2%
Minibatch loss at step 1000: 1.432582
Minibatch accuracy: 76.6%
Validation accuracy: 76.4%
Test accuracy: 84.4%
Minibatch loss at step 1500: 0.757141
Minibatch accuracy: 79.7%
Validation accuracy: 77.4%
Test accuracy: 85.2%
Minibatch loss at step 2000: 0.906352
Minibatch accuracy: 81.2%
Validation accuracy: 76.8%
Test accuracy: 85.2%
Minibatch loss at step 2500: 1.114663
Minibatch accuracy: 75.0%
Validation accuracy: 77.9%
Test accuracy: 86.0%
Minibatch loss at step 3000: 0.994610
Minibatch accuracy: 77.3%
Validation accuracy: 78.5%
Test accuracy: 86.3%

>>>>>>>>>> Beta: 0.001000%
Initialized
Minibatch loss at step 0: 20.761656
Minibatch accuracy: 10.2%
Validation accuracy: 10.4%
Test accuracy: 11.3%
Minibatch loss at step 500: 1

In [7]:
print("*** Best beta:"+str(betas[np.argmax(test_accuracy)])+ " -- accuracy:" + str(test_accuracy[np.argmax(test_accuracy)]))

*** Best beta:0.1 -- accuracy:88.87


We got an __improvement in test accuracy__ vs. not regularized model (~86.3%). 

### Neural Networks models: 1 hidden layer 

In [8]:
##### nn model 
import math

def create_nn1_model_and_run(graph,
                         train_dataset,
                         train_labels,
                         valid_dataset,
                         valid_labels,
                         test_dataset,
                         test_labels,
                         beta,
                         num_steps,
                         hidden_size = 1024, 
                         num_labels=10,batch_size = 128):
    
    uniMax = 1/math.sqrt(hidden_size)
    
    with graph.as_default():
      tf_train_dataset = tf.placeholder(tf.float32,shape=(batch_size, image_size * image_size))
      tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
        
      tf_valid_dataset = tf.constant(valid_dataset)
      tf_test_dataset = tf.constant(test_dataset)

      # Hidden 1
      weights_1 = tf.Variable(tf.random_uniform([image_size * image_size, hidden_size], minval=-uniMax, maxval=uniMax),
                             name='weights_1')
      biases_1 = tf.Variable(tf.random_uniform([hidden_size],minval=-uniMax, maxval=uniMax),name='biases_1')
      hidden_1 = tf.nn.relu(tf.matmul(tf_train_dataset, weights_1) + biases_1)

      # Softmax 
      weights_2 = tf.Variable(tf.random_uniform([hidden_size, num_labels],minval=-uniMax, maxval=uniMax), name='weights_2')
      biases_2 = tf.Variable(tf.random_uniform([num_labels],minval=-uniMax, maxval=uniMax),name='biases_2')
      logits = tf.matmul(hidden_1, weights_2) + biases_2

      # 
      loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))+(1/batch_size)*beta*(tf.nn.l2_loss(weights_1)+tf.nn.l2_loss(weights_2)) 

      # Optimizer
      global_step = tf.Variable(0)  # count the number of steps taken.
      learning_rate = tf.train.exponential_decay(0.5, global_step, 100000, 0.96, staircase=True)
      optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
      #optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)

      # Predictions for the training, validation, and test data.
      train_prediction = tf.nn.softmax(logits)
      valid_prediction = tf.nn.softmax(
        tf.matmul(tf.nn.relu(tf.matmul(tf_valid_dataset, weights_1) + biases_1), weights_2) + biases_2)
      test_prediction = tf.nn.softmax(
        tf.matmul(tf.nn.relu(tf.matmul(tf_test_dataset, weights_1) + biases_1), weights_2) + biases_2)

    test_accuracy = 0
    with tf.Session(graph=graph) as session:
        tf.global_variables_initializer().run()
        print("Initialized")
        for step in range(num_steps):
            # Pick an offset within the training data, which has been randomized.
            # Note: we could use better randomization across epochs.
            offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
            # Generate a minibatch.
            batch_data = train_dataset[offset:(offset + batch_size), :]
            batch_labels = train_labels[offset:(offset + batch_size), :]
            # Prepare a dictionary telling the session where to feed the minibatch.
            # The key of the dictionary is the placeholder node of the graph to be fed,
            # and the value is the numpy array to feed to it.
            feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
            _, l, predictions = session.run(
              [optimizer, loss, train_prediction], feed_dict=feed_dict)
            if (step % 500 == 0):
              print("Minibatch loss at step %d: %f" % (step, l))
              print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
              print("Validation accuracy: %.1f%%" % accuracy(
              valid_prediction.eval(), valid_labels))
              test_accuracy = accuracy(test_prediction.eval(), test_labels)
              print("Test accuracy: %.1f%%" % test_accuracy)
    return test_accuracy

In [9]:
betas = [0, 0.001,0.01,0.1,1,10]
test_accuracy = np.zeros(len(betas))
i = 0
for beta in betas:
  print("\n>>>>>>>>>> Beta: %f%%" % beta)
  graph = tf.Graph()
  test_accuracy[i] = create_nn1_model_and_run(graph,
                         train_dataset,
                         train_labels,
                         valid_dataset,
                         valid_labels,
                         test_dataset,
                         test_labels,
                         beta,
                         num_steps)
   
  i = i +1


>>>>>>>>>> Beta: 0.000000%
Initialized
Minibatch loss at step 0: 2.297858
Minibatch accuracy: 18.0%
Validation accuracy: 36.9%
Test accuracy: 40.3%
Minibatch loss at step 500: 0.367253
Minibatch accuracy: 90.6%
Validation accuracy: 85.6%
Test accuracy: 91.9%
Minibatch loss at step 1000: 0.526298
Minibatch accuracy: 85.2%
Validation accuracy: 86.4%
Test accuracy: 92.8%
Minibatch loss at step 1500: 0.276079
Minibatch accuracy: 93.8%
Validation accuracy: 87.8%
Test accuracy: 93.9%
Minibatch loss at step 2000: 0.284038
Minibatch accuracy: 93.8%
Validation accuracy: 87.8%
Test accuracy: 94.0%
Minibatch loss at step 2500: 0.351037
Minibatch accuracy: 89.8%
Validation accuracy: 88.0%
Test accuracy: 94.2%
Minibatch loss at step 3000: 0.346995
Minibatch accuracy: 89.1%
Validation accuracy: 88.3%
Test accuracy: 94.3%

>>>>>>>>>> Beta: 0.001000%
Initialized
Minibatch loss at step 0: 2.324186
Minibatch accuracy: 5.5%
Validation accuracy: 40.0%
Test accuracy: 43.3%
Minibatch loss at step 500: 0.35

In [10]:
print("*** Best beta:"+str(betas[np.argmax(test_accuracy)])+ " -- accuracy:" + str(test_accuracy[np.argmax(test_accuracy)]))

*** Best beta:0.01 -- accuracy:94.42


We got an __improvement in test accuracy__ vs. not regularized model (~89.1%). 

### Neural Networks models: 2 hidden layers 

In [20]:
def create_nn2_model_and_run(graph,
                         train_dataset,
                         train_labels,
                         valid_dataset,
                         valid_labels,
                         test_dataset,
                         test_labels,
                         beta,
                         num_steps,
                         hidden_size = 1024, 
                         num_labels=10,batch_size = 128):
    
    uniMax = 1/math.sqrt(hidden_size)
    
    with graph.as_default():
      # Input data. For the training data, we use a placeholder that will be fed
      # at run time with a training minibatch.
      tf_train_dataset = tf.placeholder(tf.float32,shape=(batch_size, image_size * image_size))
      tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
        
      tf_valid_dataset = tf.constant(valid_dataset)
      tf_test_dataset = tf.constant(test_dataset)

      # Hidden 1
      weights_1 = tf.Variable(tf.random_uniform([image_size * image_size, hidden_size], minval=-uniMax, maxval=uniMax),
                             name='weights_1')
      biases_1 = tf.Variable(tf.random_uniform([hidden_size],minval=-uniMax, maxval=uniMax),name='biases_1')
      hidden_1 = tf.nn.relu(tf.matmul(tf_train_dataset, weights_1) + biases_1)
        
      # Hidden 2
      weights_2 = tf.Variable(tf.random_uniform([hidden_size, hidden_size], minval=-uniMax, maxval=uniMax),name='weights_2')
      biases_2 = tf.Variable(tf.random_uniform([hidden_size],minval=-uniMax, maxval=uniMax),name='biases_2')
      hidden_2 = tf.nn.relu(tf.matmul(hidden_1, weights_2) + biases_2)

      # Softmax 
      weights_3 = tf.Variable(tf.random_uniform([hidden_size, num_labels],minval=-uniMax, maxval=uniMax), name='weights_3')
      biases_3 = tf.Variable(tf.random_uniform([num_labels],minval=-uniMax, maxval=uniMax),name='biases_3')
      logits = tf.matmul(hidden_2, weights_3) + biases_3

      # 
      loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels) )+(1/batch_size)*beta*(tf.nn.l2_loss(weights_1)+tf.nn.l2_loss(weights_2)+tf.nn.l2_loss(weights_3))

      # Optimizer.
      global_step = tf.Variable(0)  # count the number of steps taken.
      learning_rate = tf.train.exponential_decay(0.5, global_step, 100000, 0.96, staircase=True)
      optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
      #optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)

      # Predictions for the training, validation, and test data.
      train_prediction = tf.nn.softmax(logits)
      
      valid_prediction = tf.nn.softmax(
        tf.matmul(tf.nn.relu(tf.matmul(tf.nn.relu(tf.matmul(tf_valid_dataset, weights_1) + biases_1), weights_2) + biases_2),
              weights_3)+biases_3)
      test_prediction = tf.nn.softmax(
            tf.matmul(tf.nn.relu(tf.matmul(tf.nn.relu(tf.matmul(tf_test_dataset, weights_1) + biases_1), weights_2) + biases_2),
              weights_3)+biases_3)

    test_accuracy = 0
    with tf.Session(graph=graph) as session:
        tf.global_variables_initializer().run()
        print("Initialized")
        for step in range(num_steps):
    
            offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
            
            # Generate a minibatch.
            batch_data = train_dataset[offset:(offset + batch_size), :]
            batch_labels = train_labels[offset:(offset + batch_size), :]
            
            feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
            _, l, predictions = session.run(
              [optimizer, loss, train_prediction], feed_dict=feed_dict)
            
            if (step % 500 == 0):
              print("Minibatch loss at step %d: %f" % (step, l))
              print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
              print("Validation accuracy: %.1f%%" % accuracy(
              valid_prediction.eval(), valid_labels))
              test_accuracy = accuracy(test_prediction.eval(), test_labels)
              print("Test accuracy: %.1f%%" % test_accuracy)
    return test_accuracy

In [21]:
betas = [0, 0.001,0.01,0.1,1,10]
test_accuracy = np.zeros(len(betas))
i = 0
for beta in betas:
  print("\n>>>>>>>>>> Beta: %f%%" % beta)
  graph = tf.Graph()
  test_accuracy[i] = create_nn2_model_and_run(graph,
                         train_dataset,
                         train_labels,
                         valid_dataset,
                         valid_labels,
                         test_dataset,
                         test_labels,
                         beta,
                         num_steps)
   
  i = i +1


>>>>>>>>>> Beta: 0.000000%
Initialized
Minibatch loss at step 0: 2.297088
Minibatch accuracy: 14.1%
Validation accuracy: 35.2%
Test accuracy: 38.1%
Minibatch loss at step 500: 0.349590
Minibatch accuracy: 89.1%
Validation accuracy: 85.7%
Test accuracy: 92.3%
Minibatch loss at step 1000: 0.472860
Minibatch accuracy: 85.2%
Validation accuracy: 86.8%
Test accuracy: 93.0%
Minibatch loss at step 1500: 0.243931
Minibatch accuracy: 93.0%
Validation accuracy: 88.3%
Test accuracy: 94.1%
Minibatch loss at step 2000: 0.234967
Minibatch accuracy: 94.5%
Validation accuracy: 88.5%
Test accuracy: 94.6%
Minibatch loss at step 2500: 0.300683
Minibatch accuracy: 92.2%
Validation accuracy: 89.1%
Test accuracy: 94.9%
Minibatch loss at step 3000: 0.320041
Minibatch accuracy: 89.8%
Validation accuracy: 89.1%
Test accuracy: 94.9%

>>>>>>>>>> Beta: 0.001000%
Initialized
Minibatch loss at step 0: 2.306846
Minibatch accuracy: 13.3%
Validation accuracy: 17.6%
Test accuracy: 18.9%
Minibatch loss at step 500: 0.3

In [13]:
print("*** Best beta:"+str(betas[np.argmax(test_accuracy)])+ " -- accuracy:" + str(test_accuracy[np.argmax(test_accuracy)]))

*** Best beta:0 -- accuracy:94.47


__We did not get a significative improvement in test accuracy__ vs. not regularized model (~94.7%). 


## Neural Networks models: 3 hidden layers

In [26]:
def create_nn3_model_and_run(graph,
                         train_dataset,
                         train_labels,
                         valid_dataset,
                         valid_labels,
                         test_dataset,
                         test_labels,
                         beta,
                         num_steps,
                         hidden_size = 1024, 
                         num_labels=10,batch_size = 128):
    
    uniMax = 1/math.sqrt(hidden_size)
    
    with graph.as_default():
      # Input data. For the training data, we use a placeholder that will be fed
      # at run time with a training minibatch.
      tf_train_dataset = tf.placeholder(tf.float32,shape=(batch_size, image_size * image_size))
      tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
        
      tf_valid_dataset = tf.constant(valid_dataset)
      tf_test_dataset = tf.constant(test_dataset)

      # Hidden 1
      weights_1 = tf.Variable(tf.random_uniform([image_size * image_size, hidden_size], minval=-uniMax, maxval=uniMax),
                             name='weights_1')
      biases_1 = tf.Variable(tf.random_uniform([hidden_size],minval=-uniMax, maxval=uniMax),name='biases_1')
      hidden_1 = tf.nn.relu(tf.matmul(tf_train_dataset, weights_1) + biases_1)
        
      # Hidden 2
      weights_2 = tf.Variable(tf.random_uniform([hidden_size, hidden_size], minval=-uniMax, maxval=uniMax),name='weights_2')
      biases_2 = tf.Variable(tf.random_uniform([hidden_size],minval=-uniMax, maxval=uniMax),name='biases_2')
      hidden_2 = tf.nn.relu(tf.matmul(hidden_1, weights_2) + biases_2)
    
      # Hidden 3
      weights_3 = tf.Variable(tf.random_uniform([hidden_size, hidden_size], minval=-uniMax, maxval=uniMax),name='weights_3')
      biases_3 = tf.Variable(tf.random_uniform([hidden_size],minval=-uniMax, maxval=uniMax),name='biases_3')
      hidden_3 = tf.nn.relu(tf.matmul(hidden_2, weights_3) + biases_3)

      # Softmax 
      weights_4 = tf.Variable(tf.random_uniform([hidden_size, num_labels],minval=-uniMax, maxval=uniMax), name='weights_4')
      biases_4 = tf.Variable(tf.random_uniform([num_labels],minval=-uniMax, maxval=uniMax),name='biases_4')
      logits = tf.matmul(hidden_3, weights_4) + biases_4

      # 
      loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels) )+(1/batch_size)*beta*(tf.nn.l2_loss(weights_1)+tf.nn.l2_loss(weights_2)+tf.nn.l2_loss(weights_3)+tf.nn.l2_loss(weights_4))

      # Optimizer.
      global_step = tf.Variable(0)  # count the number of steps taken.
      learning_rate = tf.train.exponential_decay(0.5, global_step, 100000, 0.96, staircase=True)
      optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
      #optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)

      # Predictions for the training, validation, and test data.
      train_prediction = tf.nn.softmax(logits)
      
      valid_prediction = tf.nn.softmax(
        tf.matmul(tf.nn.relu(tf.matmul(tf.nn.relu(tf.matmul(tf.nn.relu(tf.matmul(tf_valid_dataset, weights_1) + biases_1), weights_2) + biases_2),
              weights_3)+biases_3),weights_4)+biases_4)
      test_prediction = tf.nn.softmax(
            tf.matmul(tf.nn.relu(tf.matmul(tf.nn.relu(tf.matmul(tf.nn.relu(tf.matmul(tf_test_dataset, weights_1) + biases_1), weights_2) + biases_2),
              weights_3)+biases_3),weights_4)+biases_4)

    test_accuracy = 0
    with tf.Session(graph=graph) as session:
        tf.global_variables_initializer().run()
        print("Initialized")
        for step in range(num_steps):
    
            offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
            
            # Generate a minibatch.
            batch_data = train_dataset[offset:(offset + batch_size), :]
            batch_labels = train_labels[offset:(offset + batch_size), :]
            
            feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
            _, l, predictions = session.run(
              [optimizer, loss, train_prediction], feed_dict=feed_dict)
            
            if (step % 500 == 0):
              print("Minibatch loss at step %d: %f" % (step, l))
              print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
              print("Validation accuracy: %.1f%%" % accuracy(
              valid_prediction.eval(), valid_labels))
              test_accuracy = accuracy(test_prediction.eval(), test_labels)
              print("Test accuracy: %.1f%%" % test_accuracy)
    return test_accuracy

In [27]:
betas = [0, 0.001,0.01,0.1,1,10]
test_accuracy = np.zeros(len(betas))
i = 0
for beta in betas:
  print("\n>>>>>>>>>> Beta: %f%%" % beta)
  graph = tf.Graph()
  test_accuracy[i] = create_nn3_model_and_run(graph,
                         train_dataset,
                         train_labels,
                         valid_dataset,
                         valid_labels,
                         test_dataset,
                         test_labels,
                         beta,
                         num_steps)
   
  i = i +1


>>>>>>>>>> Beta: 0.000000%
Initialized
Minibatch loss at step 0: 2.300773
Minibatch accuracy: 12.5%
Validation accuracy: 18.2%
Test accuracy: 19.5%
Minibatch loss at step 500: 0.354965
Minibatch accuracy: 88.3%
Validation accuracy: 85.5%
Test accuracy: 92.1%
Minibatch loss at step 1000: 0.463847
Minibatch accuracy: 84.4%
Validation accuracy: 86.4%
Test accuracy: 92.8%
Minibatch loss at step 1500: 0.258584
Minibatch accuracy: 91.4%
Validation accuracy: 88.2%
Test accuracy: 93.9%
Minibatch loss at step 2000: 0.250371
Minibatch accuracy: 93.8%
Validation accuracy: 88.4%
Test accuracy: 94.8%
Minibatch loss at step 2500: 0.302076
Minibatch accuracy: 90.6%
Validation accuracy: 88.9%
Test accuracy: 94.8%
Minibatch loss at step 3000: 0.355451
Minibatch accuracy: 88.3%
Validation accuracy: 88.9%
Test accuracy: 95.1%

>>>>>>>>>> Beta: 0.001000%
Initialized
Minibatch loss at step 0: 2.306205
Minibatch accuracy: 10.9%
Validation accuracy: 19.2%
Test accuracy: 20.4%
Minibatch loss at step 500: 0.3

In [28]:
print("*** Best beta:"+str(betas[np.argmax(test_accuracy)])+ " -- accuracy:" + str(test_accuracy[np.argmax(test_accuracy)]))

*** Best beta:0 -- accuracy:95.1


__We did not get a significative improvement in test accuracy__ vs. not regularized model (~95.0%). 

### Conclusions

* __Logistic Model__: __L2 regularization effective__
    * unregularized - test accuracy 86.3%
    * regularized (best beta:0.1) - test accuracy:88.87%
* __Neural Networks models, 1 hidden layer__: __L2 regularization effective__
    * unregularized - test accuracy 89.1%
    * regularized (best beta:0.01) - test accuracy:94.42%
* __Neural Networks models, 2 hidden layers__: __L2 regularization not effective__
    * unregularized - test accuracy 94.7%
    * regularized (best beta:0, i.e. __unregularized__) - test accuracy:94.47%
* __Neural Networks models, 3 hidden layers__: __L2 regularization not effective__
    * unregularized - test accuracy 95.0%
    * regularized (best beta:0, i.e. __unregularized__) - test accuracy:95.1%
        
In general we note that L2 regularization is less effective for deep learning architectures with higher number of hidden layers